In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector,Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.utils import to_categorical
%matplotlib inline

ModuleNotFoundError: No module named 'pandas'

In [271]:
#normalize
def normalize(df):
    norm = df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
    return norm
#scale back to the true range
def true_range(df, x):
    X = (x * (df['mid1'].max() - df['mid1'].min())) + df['mid1'].min()
    return X
#rolling windows
def train_windows(df, ref_day=10, predict_day=5):
    X_train, Y_train = [], []
    for i in range(int(df.shape[0] / 5 - 2)):
        #扣掉response
        #記得最後一行要放response
        #if i == df.shape[0]-predict_day-ref_day -1:
        #    X_train.append(np.array(df.iloc[i:i+ref_day,:-1]))
        #    Y_train.append(np.array(df.iloc[i+ref_day:]["mid1"]))
        #else:  
        X_train.append(np.array(df.iloc[i * predict_day: predict_day * (i + 2),:-1]))
        Y_train.append(np.array(df.iloc[(i+2) * predict_day: (i+3) * predict_day]["mid1"]))
    return np.array(X_train), np.array(Y_train)
#LSTM (when add the drop out layer in order to deal with overfitting)
def lstm_stock_model(shape):
    model = Sequential()
    #return sequence = True -> means the network will have long memory
    model.add(LSTM(256, input_shape=(shape[1], shape[2]), return_sequences=True))
    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(1)))
    model.add(Flatten())
    #10 -> X shape
    model.add(Dense(10,activation='linear'))
    #5 -> Y shape
    model.add(Dense(5,activation='linear'))
    model.compile(loss="mean_squared_error", optimizer="adam",metrics=['mean_squared_error'])
    model.summary()
    return model

#record transaction
def record_transaction(para, date, tick, action_side, price, equity_value, transaction_history):
    info = {
                'Date': date,
                'tick': tick,
                'Action': action_side,
                'Price': price,
                'Equity_value': equity_value
            }
    if para in transaction_history:
        transaction_history[para].append(info)
    else:
        transaction_history.update({para: [info]})    
    
    return transaction_history

#performance
def performance(equity_value_, transaction):
    #return
    Return = equity_value_.iloc[-1][0]
    print('return:', Return)
    #vol
    Std = equity_value_.std()[0]
    print('std:', Std)
    #sharp ratio
    Sharp_ratio = (Return - 0.0052) / Std
    print('sharp ratio:', Sharp_ratio)
    #nums of trading
    Trades = len(transaction) / 2
    print('trades:', Trades)
    #odds ratio
    R = 0
    for i in range(len(transaction)):
        if transaction[i]['Equity_value'] > 0:
            R += 1
    Odd = R / (len(transaction) / 2)
    print('odds ratio:', Odd)
    #mdd
    Mdd = (equity_value_.cummax() - equity_value_).max()[0]
    print('mdd:', Mdd)
    #skew
    Skew = equity_value_.skew()[0]
    print('skewness:', Skew)
     
    return Return, Std, Sharp_ratio, Trades, Odd, Mdd, Skew

#model performance
def model_performance(y, f):
    tot = np.sum(np.square(y - np.mean(y)))
    res = np.sum(np.square(y - f))
    r_squared = 1 - res / tot
    mse = np.sum(np.square(y - f)) * (1 / len(y))
    print('MSE:', mse)
    print('r-squared:', r_squared)

In [267]:
df = pd.read_csv('./data/3406.csv')
df['spread1'] = df['Ask1'] - df['Bid1']
df['spread2'] = df['Ask2'] - df['Bid2']
df['spread3'] = df['Ask3'] - df['Bid3']
df['spread4'] = df['Ask4'] - df['Bid4']
df['spread5'] = df['Ask5'] - df['Bid5']
df['mid2'] = (df['Ask2'] + df['Bid2']) / 2
df['mid3'] = (df['Ask3'] + df['Bid3']) / 2
df['mid4'] = (df['Ask4'] + df['Bid4']) / 2
df['mid5'] = (df['Ask5'] + df['Bid5']) / 2
df['a51_diff'] = df['Ask5'] - df['Ask1']
df['b15_diff'] = df['Bid1'] - df['Bid5']
df['a21_diff_abs'] = np.abs(df['Ask2'] - df['Ask1'])
df['a32_diff_abs'] = np.abs(df['Ask3'] - df['Ask2'])
df['a43_diff_abs'] = np.abs(df['Ask4'] - df['Ask3'])
df['a54_diff_abs'] = np.abs(df['Ask5'] - df['Ask4'])
df['b21_diff_abs'] = np.abs(df['Bid2'] - df['Bid1'])
df['b32_diff_abs'] = np.abs(df['Bid3'] - df['Bid2'])
df['b43_diff_abs'] = np.abs(df['Bid4'] - df['Bid3'])
df['b54_diff_abs'] = np.abs(df['Bid5'] - df['Bid4'])
df['a_mean'] = (df['Ask1'] +df['Ask2'] +df['Ask3'] +df['Ask4'] +df['Ask5']) / 5
df['b_mean'] = (df['Bid1'] +df['Bid2'] +df['Bid3'] +df['Bid4'] +df['Bid5']) / 5
df['aq_mean'] = (df['AskQty1'] +df['AskQty2'] +df['AskQty3'] +df['AskQty4'] +df['AskQty5']) / 5
df['bq_mean'] = (df['BidQty1'] +df['BidQty2'] +df['BidQty3'] +df['BidQty4'] +df['BidQty5']) / 5
df['p_acc_diff'] = (df['Ask1'] - df['Bid1']) +(df['Ask2'] - df['Bid2']) + (df['Ask3'] - df['Bid3']) + (df['Ask4'] - df['Bid4']) + (df['Ask5'] - df['Bid5'])
df['q_acc_diff'] = (df['AskQty1'] - df['BidQty1']) +(df['AskQty2'] - df['BidQty2']) + (df['AskQty3'] - df['BidQty3']) + (df['AskQty4'] - df['BidQty4']) + (df['AskQty5'] - df['BidQty5'])
#1 tick
df['da1_dt_1'] = df['Ask1'] - df['Ask1'].shift(1)
df['da2_dt_1'] = df['Ask2'] - df['Ask2'].shift(1)
df['da3_dt_1'] = df['Ask3'] - df['Ask3'].shift(1)
df['da4_dt_1'] = df['Ask4'] - df['Ask4'].shift(1)
df['da5_dt_1'] = df['Ask5'] - df['Ask5'].shift(1)
df['db1_dt_1'] = df['Bid1'] - df['Bid1'].shift(1)
df['db2_dt_1'] = df['Bid2'] - df['Bid2'].shift(1)
df['db3_dt_1'] = df['Bid3'] - df['Bid3'].shift(1)
df['db4_dt_1'] = df['Bid4'] - df['Bid4'].shift(1)
df['db5_dt_1'] = df['Bid5'] - df['Bid5'].shift(1)
#5 ticks
df['da1_dt_5'] = df['Ask1'] - df['Ask1'].shift(5)
df['da2_dt_5'] = df['Ask2'] - df['Ask2'].shift(5)
df['da3_dt_5'] = df['Ask3'] - df['Ask3'].shift(5)
df['da4_dt_5'] = df['Ask4'] - df['Ask4'].shift(5)
df['da5_dt_5'] = df['Ask5'] - df['Ask5'].shift(5)
df['db1_dt_5'] = df['Bid1'] - df['Bid1'].shift(5)
df['db2_dt_5'] = df['Bid2'] - df['Bid2'].shift(5)
df['db3_dt_5'] = df['Bid3'] - df['Bid3'].shift(5)
df['db4_dt_5'] = df['Bid4'] - df['Bid4'].shift(5)
df['db5_dt_5'] = df['Bid5'] - df['Bid5'].shift(5)
##10 ticks
df['da1_dt_10'] = df['Ask1'] - df['Ask1'].shift(10)
df['da2_dt_10'] = df['Ask2'] - df['Ask2'].shift(10)
df['da3_dt_10'] = df['Ask3'] - df['Ask3'].shift(10)
df['da4_dt_10'] = df['Ask4'] - df['Ask4'].shift(10)
df['da5_dt_10'] = df['Ask5'] - df['Ask5'].shift(10)
df['db1_dt_10'] = df['Bid1'] - df['Bid1'].shift(10)
df['db2_dt_10'] = df['Bid2'] - df['Bid2'].shift(10)
df['db3_dt_10'] = df['Bid3'] - df['Bid3'].shift(10)
df['db4_dt_10'] = df['Bid4'] - df['Bid4'].shift(10)
df['db5_dt_10'] = df['Bid5'] - df['Bid5'].shift(10)
#1 tick
df['daq1_dt_1'] = df['AskQty1'] - df['AskQty1'].shift(1)
df['daq2_dt_1'] = df['AskQty2'] - df['AskQty2'].shift(1)
df['daq3_dt_1'] = df['AskQty3'] - df['AskQty3'].shift(1)
df['daq4_dt_1'] = df['AskQty4'] - df['AskQty4'].shift(1)
df['daq5_dt_1'] = df['AskQty5'] - df['AskQty5'].shift(1)
df['dbq1_dt_1'] = df['BidQty1'] - df['BidQty1'].shift(1)
df['dbq2_dt_1'] = df['BidQty2'] - df['BidQty2'].shift(1)
df['dbq3_dt_1'] = df['BidQty3'] - df['BidQty3'].shift(1)
df['dbq4_dt_1'] = df['BidQty4'] - df['BidQty4'].shift(1)
df['dbq5_dt_1'] = df['BidQty5'] - df['BidQty5'].shift(1)
#5 ticks
df['daq1_dt_5'] = df['AskQty1'] - df['AskQty1'].shift(5)
df['daq2_dt_5'] = df['AskQty2'] - df['AskQty2'].shift(5)
df['daq3_dt_5'] = df['AskQty3'] - df['AskQty3'].shift(5)
df['daq4_dt_5'] = df['AskQty4'] - df['AskQty4'].shift(5)
df['daq5_dt_5'] = df['AskQty5'] - df['AskQty5'].shift(5)
df['dbq1_dt_5'] = df['BidQty1'] - df['BidQty1'].shift(5)
df['dbq2_dt_5'] = df['BidQty2'] - df['BidQty2'].shift(5)
df['dbq3_dt_5'] = df['BidQty3'] - df['BidQty3'].shift(5)
df['dbq4_dt_5'] = df['BidQty4'] - df['BidQty4'].shift(5)
df['dbq5_dt_5'] = df['BidQty5'] - df['BidQty5'].shift(5)
#10 ticks
df['daq1_dt_10'] = df['AskQty1'] - df['AskQty1'].shift(10)
df['daq2_dt_10'] = df['AskQty2'] - df['AskQty2'].shift(10)
df['daq3_dt_10'] = df['AskQty3'] - df['AskQty3'].shift(10)
df['daq4_dt_10'] = df['AskQty4'] - df['AskQty4'].shift(10)
df['daq5_dt_10'] = df['AskQty5'] - df['AskQty5'].shift(10)
df['dbq1_dt_10'] = df['BidQty1'] - df['BidQty1'].shift(10)
df['dbq2_dt_10'] = df['BidQty2'] - df['BidQty2'].shift(10)
df['dbq3_dt_10'] = df['BidQty3'] - df['BidQty3'].shift(10)
df['dbq4_dt_10'] = df['BidQty4'] - df['BidQty4'].shift(10)
df['dbq5_dt_10'] = df['BidQty5'] - df['BidQty5'].shift(10)
#response 放在最後一行
df['mid1'] = (df['Ask1'] + df['Bid1']) / 2
#drop nan
df = df.dropna()
df = df.reset_index(drop = True)
#b43_diff_abs and b54_diff_abs are all same values -> drop
#df = df.drop(['b43_diff_abs'], axis = 1)
#df = df.drop(['b54_diff_abs'], axis = 1)
from datetime import datetime
df['TxTime'] = pd.to_datetime(df['TxTime'])
df['TxTime'] = [str(x)[0:10] for x in df['TxTime']]
#some index's ask1 = 0, we change that mid1 to bid1
idx = df[df['Ask1'] == 0].index
for i in idx:
    df['mid1'].iloc[i] = df['Bid1'].iloc[i]

C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [268]:
#3/23 3/24 data有缺值 -> drop
df = df[df['TxTime'] != '2020-03-23']
df = df[df['TxTime'] != '2020-03-24']
df = df.reset_index(drop = True)
#drop TxTime
df2 = df.drop(['TxTime'], axis = 1)

In [236]:
train_5t = pd.read_csv('./data/train_5t.csv')
test_5t = pd.read_csv('./data/test_5t.csv')

In [237]:
#已實現損益
df['equity_value'] = pd.DataFrame(data = np.zeros((len(df), 1)), index = df.index[:])
#未實現損益
df['unrealized'] = pd.DataFrame(data = np.zeros((len(df), 1)), index = df.index[:])
df['holding'] = pd.DataFrame(data = np.zeros((len(df), 1)), index = df.index[:])
#drop 4/15 的 last tick
df_test = df.iloc[282250: ]
#為了可以被5整除
df_train = df.iloc[0:282250]
#1~10 pred 11, hence start from 11
df_train = df_train.iloc[10:, :]
df_test = df_test.iloc[10:, :]
#reset index
df_train = df_train.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [238]:
df_train

,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,...,daq5_dt_10,dbq1_dt_10,dbq2_dt_10,dbq3_dt_10,dbq4_dt_10,dbq5_dt_10,mid1,equity_value,unrealized,holding
0,397.0,1,2,396.5,397.0,3,6,396.0,398.0,20,...,-57.0,0.0,5.0,34.0,-31.0,-8.0,396.75,0.0,0.0,0.0
1,397.0,1,2,396.5,397.0,2,6,396.0,398.0,20,...,-57.0,1.0,0.0,0.0,0.0,0.0,396.75,0.0,0.0,0.0
2,396.5,2,6,396.0,397.0,2,40,395.0,398.0,20,...,-57.0,4.0,34.0,-31.0,-8.0,5.0,396.50,0.0,0.0,0.0
3,397.0,2,6,396.0,397.0,0,40,395.0,398.0,20,...,0.0,4.0,34.0,-31.0,-8.0,5.0,396.50,0.0,0.0,0.0
4,398.0,20,6,396.0,397.0,0,40,395.0,398.0,20,...,0.0,4.0,34.0,-31.0,-8.0,5.0,396.50,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282235,0.0,0,15,460.5,461.0,22,16,460.0,461.5,11,...,0.0,13.0,-3.0,0.0,0.0,0.0,460.75,0.0,0.0,0.0
282236,0.0,0,16,460.5,461.0,22,16,460.0,461.5,11,...,0.0,14.0,-2.0,0.0,0.0,0.0,460.75,0.0,0.0,0.0
282237,0.0,0,16,460.5,461.0,21,16,460.0,461.5,11,...,0.0,14.0,-2.0,0.0,0.0,0.0,460.75,0.0,0.0,0.0
282238,460.5,1,15,460.5,461.0,21,16,460.0,461.5,11,...,0.0,13.0,-2.0,0.0,0.0,0.0,460.75,0.0,0.0,0.0


In [239]:
#calculate pred return
df_train['pred_mid'] = train_5t[['0']]
#df_train['mid_shift'] = df_train['pred_mid'].shift(1)
df_test['pred_mid'] = test_5t[['0']]

In [240]:
df_train

,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,...,dbq1_dt_10,dbq2_dt_10,dbq3_dt_10,dbq4_dt_10,dbq5_dt_10,mid1,equity_value,unrealized,holding,pred_mid
0,397.0,1,2,396.5,397.0,3,6,396.0,398.0,20,...,0.0,5.0,34.0,-31.0,-8.0,396.75,0.0,0.0,0.0,395.67432
1,397.0,1,2,396.5,397.0,2,6,396.0,398.0,20,...,1.0,0.0,0.0,0.0,0.0,396.75,0.0,0.0,0.0,397.20070
2,396.5,2,6,396.0,397.0,2,40,395.0,398.0,20,...,4.0,34.0,-31.0,-8.0,5.0,396.50,0.0,0.0,0.0,397.32635
3,397.0,2,6,396.0,397.0,0,40,395.0,398.0,20,...,4.0,34.0,-31.0,-8.0,5.0,396.50,0.0,0.0,0.0,396.99033
4,398.0,20,6,396.0,397.0,0,40,395.0,398.0,20,...,4.0,34.0,-31.0,-8.0,5.0,396.50,0.0,0.0,0.0,394.36914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282235,0.0,0,15,460.5,461.0,22,16,460.0,461.5,11,...,13.0,-3.0,0.0,0.0,0.0,460.75,0.0,0.0,0.0,461.53134
282236,0.0,0,16,460.5,461.0,22,16,460.0,461.5,11,...,14.0,-2.0,0.0,0.0,0.0,460.75,0.0,0.0,0.0,461.55368
282237,0.0,0,16,460.5,461.0,21,16,460.0,461.5,11,...,14.0,-2.0,0.0,0.0,0.0,460.75,0.0,0.0,0.0,462.08215
282238,460.5,1,15,460.5,461.0,21,16,460.0,461.5,11,...,13.0,-2.0,0.0,0.0,0.0,460.75,0.0,0.0,0.0,461.62875


In [241]:
df_test

,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,...,dbq1_dt_10,dbq2_dt_10,dbq3_dt_10,dbq4_dt_10,dbq5_dt_10,mid1,equity_value,unrealized,holding,pred_mid
0,0.0,0,2,457.5,458.0,10,9,457.0,458.5,5,...,-13.0,-7.0,7.0,38.0,-3.0,457.75,0.0,0.0,0.0,459.80307
1,0.0,0,2,457.5,458.0,10,10,457.0,458.5,5,...,-8.0,-4.0,-27.0,38.0,-11.0,457.75,0.0,0.0,0.0,459.23242
2,457.5,1,1,457.5,458.0,10,10,457.0,458.5,5,...,-10.0,-4.0,-27.0,38.0,-11.0,457.75,0.0,0.0,0.0,459.30420
3,0.0,0,1,457.5,458.0,12,10,457.0,458.5,5,...,0.0,-1.0,0.0,0.0,0.0,457.75,0.0,0.0,0.0,459.71457
4,0.0,0,1,457.5,458.0,12,10,457.0,458.5,5,...,-1.0,0.0,0.0,0.0,0.0,457.75,0.0,0.0,0.0,459.65445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104305,443.5,2,8,443.5,444.0,8,24,443.0,444.5,23,...,-17.0,14.0,-9.0,10.0,-11.0,443.75,0.0,0.0,0.0,444.16850
104306,443.5,1,7,443.5,444.0,8,24,443.0,444.5,23,...,-18.0,14.0,-9.0,11.0,-11.0,443.75,0.0,0.0,0.0,444.64813
104307,0.0,0,7,443.5,444.0,8,24,443.0,444.5,23,...,6.0,-1.0,0.0,0.0,0.0,443.75,0.0,0.0,0.0,444.64297
104308,443.5,3,4,443.5,444.0,8,24,443.0,444.5,23,...,-2.0,-1.0,0.0,0.0,0.0,443.75,0.0,0.0,0.0,444.40260


In [242]:
df_train['mid_5_shift'] = pd.DataFrame(data = np.zeros((len(df_train), 1)), index = df_train.index[:])
for i in range(int(int(len(df_train)) / 5)):
    df_train['mid_5_shift'].iloc[5 * i] = df_train['pred_mid'].iloc[(5 * i) + 4]
df_train['pred_return'] = df_train['mid_5_shift'] / df_train['pred_mid'] - 1
for i in range(len(df_train)):
    if df_train['pred_return'].iloc[i] == -1:
        df_train['pred_return'].iloc[i] = 0
df_train

C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,...,dbq3_dt_10,dbq4_dt_10,dbq5_dt_10,mid1,equity_value,unrealized,holding,pred_mid,mid_5_shift,pred_return
0,397.0,1,2,396.5,397.0,3,6,396.0,398.0,20,...,34.0,-31.0,-8.0,396.75,0.0,0.0,0.0,395.67432,394.36914,-0.003299
1,397.0,1,2,396.5,397.0,2,6,396.0,398.0,20,...,0.0,0.0,0.0,396.75,0.0,0.0,0.0,397.20070,0.00000,0.000000
2,396.5,2,6,396.0,397.0,2,40,395.0,398.0,20,...,-31.0,-8.0,5.0,396.50,0.0,0.0,0.0,397.32635,0.00000,0.000000
3,397.0,2,6,396.0,397.0,0,40,395.0,398.0,20,...,-31.0,-8.0,5.0,396.50,0.0,0.0,0.0,396.99033,0.00000,0.000000
4,398.0,20,6,396.0,397.0,0,40,395.0,398.0,20,...,-31.0,-8.0,5.0,396.50,0.0,0.0,0.0,394.36914,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282235,0.0,0,15,460.5,461.0,22,16,460.0,461.5,11,...,0.0,0.0,0.0,460.75,0.0,0.0,0.0,461.53134,460.52576,-0.002179
282236,0.0,0,16,460.5,461.0,22,16,460.0,461.5,11,...,0.0,0.0,0.0,460.75,0.0,0.0,0.0,461.55368,0.00000,0.000000
282237,0.0,0,16,460.5,461.0,21,16,460.0,461.5,11,...,0.0,0.0,0.0,460.75,0.0,0.0,0.0,462.08215,0.00000,0.000000
282238,460.5,1,15,460.5,461.0,21,16,460.0,461.5,11,...,0.0,0.0,0.0,460.75,0.0,0.0,0.0,461.62875,0.00000,0.000000


In [243]:
df_test['mid_5_shift'] = pd.DataFrame(data = np.zeros((len(df_test), 1)), index = df_test.index[:])
for i in range(int(int(len(df_test)) / 5)):
    df_test['mid_5_shift'].iloc[5 * i] = df_test['pred_mid'].iloc[(5 * i) + 4]
df_test['pred_return'] = df_test['mid_5_shift'] / df_test['pred_mid'] - 1
for i in range(len(df_test)):
    if df_test['pred_return'].iloc[i] == -1:
        df_test['pred_return'].iloc[i] = 0
df_test

C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,...,dbq3_dt_10,dbq4_dt_10,dbq5_dt_10,mid1,equity_value,unrealized,holding,pred_mid,mid_5_shift,pred_return
0,0.0,0,2,457.5,458.0,10,9,457.0,458.5,5,...,7.0,38.0,-3.0,457.75,0.0,0.0,0.0,459.80307,459.65445,-0.000323
1,0.0,0,2,457.5,458.0,10,10,457.0,458.5,5,...,-27.0,38.0,-11.0,457.75,0.0,0.0,0.0,459.23242,0.00000,0.000000
2,457.5,1,1,457.5,458.0,10,10,457.0,458.5,5,...,-27.0,38.0,-11.0,457.75,0.0,0.0,0.0,459.30420,0.00000,0.000000
3,0.0,0,1,457.5,458.0,12,10,457.0,458.5,5,...,0.0,0.0,0.0,457.75,0.0,0.0,0.0,459.71457,0.00000,0.000000
4,0.0,0,1,457.5,458.0,12,10,457.0,458.5,5,...,0.0,0.0,0.0,457.75,0.0,0.0,0.0,459.65445,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104305,443.5,2,8,443.5,444.0,8,24,443.0,444.5,23,...,-9.0,10.0,-11.0,443.75,0.0,0.0,0.0,444.16850,443.80120,-0.000827
104306,443.5,1,7,443.5,444.0,8,24,443.0,444.5,23,...,-9.0,11.0,-11.0,443.75,0.0,0.0,0.0,444.64813,0.00000,0.000000
104307,0.0,0,7,443.5,444.0,8,24,443.0,444.5,23,...,0.0,0.0,0.0,443.75,0.0,0.0,0.0,444.64297,0.00000,0.000000
104308,443.5,3,4,443.5,444.0,8,24,443.0,444.5,23,...,0.0,0.0,0.0,443.75,0.0,0.0,0.0,444.40260,0.00000,0.000000


In [244]:
#main strategy

In [245]:
#setting parameters
import itertools 
long_side, short_side, stop_1 = [0.001, 0.0015, 0.002, 0.0025, 0.003, 0.0035, 0.004], [-0.001, -0.0015, -0.002, -0.0025, -0.003, -0.0035, -0.004], [-0.015, -0.02, -0.025]
paras = list(itertools.product(long_side, short_side, stop_1))
paras

[(0.001, -0.001, -0.015),
 (0.001, -0.001, -0.02),
 (0.001, -0.001, -0.025),
 (0.001, -0.0015, -0.015),
 (0.001, -0.0015, -0.02),
 (0.001, -0.0015, -0.025),
 (0.001, -0.002, -0.015),
 (0.001, -0.002, -0.02),
 (0.001, -0.002, -0.025),
 (0.001, -0.0025, -0.015),
 (0.001, -0.0025, -0.02),
 (0.001, -0.0025, -0.025),
 (0.001, -0.003, -0.015),
 (0.001, -0.003, -0.02),
 (0.001, -0.003, -0.025),
 (0.001, -0.0035, -0.015),
 (0.001, -0.0035, -0.02),
 (0.001, -0.0035, -0.025),
 (0.001, -0.004, -0.015),
 (0.001, -0.004, -0.02),
 (0.001, -0.004, -0.025),
 (0.0015, -0.001, -0.015),
 (0.0015, -0.001, -0.02),
 (0.0015, -0.001, -0.025),
 (0.0015, -0.0015, -0.015),
 (0.0015, -0.0015, -0.02),
 (0.0015, -0.0015, -0.025),
 (0.0015, -0.002, -0.015),
 (0.0015, -0.002, -0.02),
 (0.0015, -0.002, -0.025),
 (0.0015, -0.0025, -0.015),
 (0.0015, -0.0025, -0.02),
 (0.0015, -0.0025, -0.025),
 (0.0015, -0.003, -0.015),
 (0.0015, -0.003, -0.02),
 (0.0015, -0.003, -0.025),
 (0.0015, -0.0035, -0.015),
 (0.0015, -0.0035,

In [246]:
#adding hard stop 
#main strategy (train set)
#record transaction
transaction_history1 = {}
#record equity values
equity_value_1 = pd.DataFrame()
#record each returns
each_return1 = pd.DataFrame()
#record unrealized gain
unrealized_1 = pd.DataFrame()
for l in range(len(paras)):
    df_train['side'] = pd.DataFrame(data = np.zeros((len(df_train), 1)), index = df_train.index[:])
    for j in range(len(df_train)):
        if df_train['pred_return'].iloc[j] > paras[l][0]:
            #long side
            df_train['side'].iloc[j] = 1
        elif df_train['pred_return'].iloc[j] < paras[l][1]:
            #short side
            df_train['side'].iloc[j] = -1
    date_idx = df_train['TxTime'].unique()
    #紀錄已實現
    equity_value1 = pd.DataFrame()
    #紀錄未實現
    unrealized1 = pd.DataFrame()
    for k in date_idx:
        df = df_train[df_train.TxTime == k]
        #side = 0
        for i in range(len(df)):
            #first tick
            if i == 0:
                #ask1 may be 0, which means 漲停 (買不到)
                if (df['side'].iloc[i] == 1) and (df['Ask1'].iloc[i] != 0):
                    #side = 1
                    df['holding'].iloc[i] = 1
                    #stock_price = df['Ask1'].iloc[i]
                    df['equity_value'].iloc[i] = 0
                    transaction_history1 = record_transaction(paras[l], k, i, 1, df['Ask1'].iloc[i], 0, transaction_history1)
                elif (df['side'].iloc[i] == 1) and (df['Ask1'].iloc[i] == 0):
                    side = 0
                    df['equity_value'].iloc[i] = 0
                elif df['side'].iloc[i] == -1:
                    #side = -1
                    df['holding'].iloc[i] = -1
                    #stock_price = df['Bid1'].iloc[i]
                    df['equity_value'].iloc[i] = - 0.0015
                    df['unrealized'].iloc[i] = -0.0015
                    transaction_history1 = record_transaction(paras[l], k, i, -1, df['Bid1'].iloc[i], -0.0015, transaction_history1)
                else:
                    #side = 0
                    df['holding'].iloc[i] = 0
                    df['equity_value'].iloc[i] = 0
            #last tick
            elif i == (len(df) - 1):
                if df['holding'].iloc[i-1] == 1:
                    df['equity_value'].iloc[i] = (df['Bid1'].iloc[i] / transaction_history1[paras[l]][-1]['Price'] - 1) - 0.0015
                    df['unrealized'].iloc[i] = 'realized'
                    #df['unrealized'].iloc[i] = df['Bid1'].iloc[i] / df['mid1'].iloc[i-1] - 1
                    transaction_history1 = record_transaction(paras[l], k, i, -1, df['Bid1'].iloc[i], df['equity_value'].iloc[i], transaction_history1)
                #ask1 may be 0, which means 漲停 (買不到)
                elif (df['holding'].iloc[i-1] == -1) and (df['Ask1'].iloc[i] != 0):
                    df['equity_value'].iloc[i] = (df['Ask1'].iloc[i] / transaction_history1[paras[l]][-1]['Price'] - 1) * (-1)
                    df['unrealized'].iloc[i] = 'realized'
                    #df['unrealized'].iloc[i] = (df['Ask1'].iloc[i] / df['mid1'].iloc[i-1] - 1) * (-1)
                    transaction_history1 = record_transaction(paras[l], k, i, 1, df['Ask1'].iloc[i], df['equity_value'].iloc[i], transaction_history1)
                #收盤如果漲停，空單結不掉，先用mid1結
                elif (df['holding'].iloc[i-1] == -1) and (df['Ask1'].iloc[i] == 0):
                    df['equity_value'].iloc[i] = (df['mid1'].iloc[i] / transaction_history1[paras[l]][-1]['Price'] - 1) * (-1)
                    df['unrealized'].iloc[i] = 'realized'
                    #df['unrealized'].iloc[i] = (df['Ask1'].iloc[i] / df['mid1'].iloc[i-1] - 1) * (-1)
                    transaction_history1 = record_transaction(paras[l], k, i, 1, df['mid1'].iloc[i], df['equity_value'].iloc[i], transaction_history1)
                else:
                    df['equity_value'].iloc[i] = 0
            else:
                if df['holding'].iloc[i-1] == 1:
                    #hard stop 1 -1 = 0 -> 0
                    if ((df['mid1'].iloc[i-1] / transaction_history1[paras[l]][-1]['Price'] - 1) < paras[l][2]): #or ((df['mid1'].iloc[i-1] / transaction_history1[paras[l]][-1]['Price'] - 1) > paras[l][2]):
                        df['holding'].iloc[i] = 0
                        df['equity_value'].iloc[i] = (df['Bid1'].iloc[i] / transaction_history1[paras[l]][-1]['Price'] - 1) - 0.0015
                        df['unrealized'].iloc[i] = 'realized'
                        #df['unrealized'].iloc[i] = df['Bid1'].iloc[i] / df['mid1'].iloc[i-1] - 1
                        transaction_history1 = record_transaction(paras[l], k, i, -1, df['Bid1'].iloc[i], df['equity_value'].iloc[i], transaction_history1)
                    #change side 1 -2 -> -1
                    elif (df['side'].iloc[i] == -1): #and ((df['mid1'].iloc[i-1] / transaction_history1[paras[l]][-1]['Price'] - 1) > 0.01):
                        df['holding'].iloc[i] = -1
                        df['equity_value'].iloc[i] = (df['Bid1'].iloc[i] / transaction_history1[paras[l]][-1]['Price'] - 1) - (0.0015 * 2)
                        df['unrealized'].iloc[i] = 'realized'
                        transaction_history1 = record_transaction(paras[l], k, i, -1, df['Bid1'].iloc[i], df['equity_value'].iloc[i] + 0.0015, transaction_history1)
                        transaction_history1 = record_transaction(paras[l], k, i, -1, df['Bid1'].iloc[i], -0.0015, transaction_history1)
                    else:
                        df['holding'].iloc[i] = df['holding'].iloc[i-1]
                        df['equity_value'].iloc[i] = 0
                        df['unrealized'].iloc[i] = (df['mid1'].iloc[i] / df['mid1'].iloc[i-1] - 1)
                elif df['holding'].iloc[i-1] == -1:
                    #hard stop -1 +1 -> 0
                    if (df['Ask1'].iloc[i] != 0) and ((df['mid1'].iloc[i-1] / transaction_history1[paras[l]][-1]['Price'] - 1) * (-1) < paras[l][2]): #or ((df['mid1'].iloc[i-1] / transaction_history1[paras[l]][-1]['Price'] - 1) * (-1) > paras[l][2])):
                        df['holding'].iloc[i] = 0
                        df['equity_value'].iloc[i] = (df['Ask1'].iloc[i] / transaction_history1[paras[l]][-1]['Price'] - 1) * (-1)
                        df['unrealized'].iloc[i] = 'realized'
                        #df['unrealized'].iloc[i] = (df['Ask1'].iloc[i] / df['mid1'].iloc[i-1] - 1) * (-1)
                        transaction_history1 = record_transaction(paras[l], k, i, 1, df['Ask1'].iloc[i], df['equity_value'].iloc[i], transaction_history1)
                    #change side -1 +2 -> 1
                    elif (df['Ask1'].iloc[i] != 0) and (df['side'].iloc[i] == 1): #and ((df['mid1'].iloc[i-1] / transaction_history1[paras[l]][-1]['Price'] - 1) * (-1) > 0.01):
                        df['holding'].iloc[i] = 1
                        df['equity_value'].iloc[i] = (df['Ask1'].iloc[i] / transaction_history1[paras[l]][-1]['Price'] - 1) * (-1)
                        df['unrealized'].iloc[i] = 'realized'
                        #df['unrealized'].iloc[i] = (df['Ask1'].iloc[i] / df['mid1'].iloc[i-1] - 1) * (-1)
                        transaction_history1 = record_transaction(paras[l], k, i, 1, df['Ask1'].iloc[i], df['equity_value'].iloc[i], transaction_history1)
                        transaction_history1 = record_transaction(paras[l], k, i, 1, df['Ask1'].iloc[i], 0, transaction_history1)
                    else:
                        df['holding'].iloc[i] = df['holding'].iloc[i-1]
                        df['equity_value'].iloc[i] = 0
                        df['unrealized'].iloc[i] = (df['mid1'].iloc[i] / df['mid1'].iloc[i-1] - 1) * (-1)
                elif df['holding'].iloc[i-1] == 0:
                    if (df['side'].iloc[i] == 1) and (df['Ask1'].iloc[i] != 0):
                        df['holding'].iloc[i] = 1
                        df['equity_value'].iloc[i] = 0
                        transaction_history1 = record_transaction(paras[l], k, i, 1, df['Ask1'].iloc[i], 0, transaction_history1)
                    elif df['side'].iloc[i] == -1:
                        df['holding'].iloc[i] = -1
                        df['equity_value'].iloc[i] = - 0.0015
                        df['unrealized'].iloc[i] = -0.0015
                        transaction_history1 = record_transaction(paras[l], k, i, -1, df['Bid1'].iloc[i], -0.0015, transaction_history1)
                    else:
                        df['holding'].iloc[i] = 0
                        df['equity_value'].iloc[i] = 0
        equity_value1 = pd.concat([equity_value1, df[['equity_value']]])
        unrealized1 = pd.concat([unrealized1, df['unrealized']])
    #concat
    equity_value_1 = pd.concat([equity_value_1, np.cumsum(equity_value1)], axis = 1)
    each_return1 = pd.concat([each_return1, equity_value1], axis = 1)
    unrealized_1 = pd.concat([unrealized_1, unrealized1], axis = 1)

C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\U

C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-pa

In [248]:
equity_value_1.columns = paras
equity_value_1.iloc[-1][equity_value_1.iloc[-1] > 0]

(0.001, -0.002, -0.015)     0.092445
(0.001, -0.002, -0.02)      0.080830
(0.001, -0.002, -0.025)     0.079266
(0.001, -0.0025, -0.015)    0.092487
(0.001, -0.0025, -0.02)     0.085440
                              ...   
(0.004, -0.0035, -0.02)     0.113494
(0.004, -0.0035, -0.025)    0.145361
(0.004, -0.004, -0.015)     0.119427
(0.004, -0.004, -0.02)      0.118251
(0.004, -0.004, -0.025)     0.150118
Name: 282239, Length: 117, dtype: float64

In [252]:
equity_value_1

,"(0.001, -0.001, -0.015)","(0.001, -0.001, -0.02)","(0.001, -0.001, -0.025)","(0.001, -0.0015, -0.015)","(0.001, -0.0015, -0.02)","(0.001, -0.0015, -0.025)","(0.001, -0.002, -0.015)","(0.001, -0.002, -0.02)","(0.001, -0.002, -0.025)","(0.001, -0.0025, -0.015)",...,"(0.004, -0.0025, -0.025)","(0.004, -0.003, -0.015)","(0.004, -0.003, -0.02)","(0.004, -0.003, -0.025)","(0.004, -0.0035, -0.015)","(0.004, -0.0035, -0.02)","(0.004, -0.0035, -0.025)","(0.004, -0.004, -0.015)","(0.004, -0.004, -0.02)","(0.004, -0.004, -0.025)"
0,-0.001500,-0.00150,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,...,-0.001500,-0.001500,-0.001500,-0.001500,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.001500,-0.00150,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,...,-0.001500,-0.001500,-0.001500,-0.001500,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-0.001500,-0.00150,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,...,-0.001500,-0.001500,-0.001500,-0.001500,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-0.001500,-0.00150,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,...,-0.001500,-0.001500,-0.001500,-0.001500,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.001500,-0.00150,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,-0.001500,...,-0.001500,-0.001500,-0.001500,-0.001500,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282235,-0.216691,-0.17306,-0.136701,-0.044522,-0.034536,-0.026909,0.084910,0.088481,0.086917,0.086022,...,0.064757,0.091887,0.093317,0.120140,0.11467,0.121144,0.153011,0.119427,0.125901,0.157768
282236,-0.216691,-0.17306,-0.136701,-0.044522,-0.034536,-0.026909,0.084910,0.088481,0.086917,0.086022,...,0.064757,0.091887,0.093317,0.120140,0.11467,0.121144,0.153011,0.119427,0.125901,0.157768
282237,-0.216691,-0.17306,-0.136701,-0.044522,-0.034536,-0.026909,0.084910,0.088481,0.086917,0.086022,...,0.064757,0.091887,0.093317,0.120140,0.11467,0.121144,0.153011,0.119427,0.125901,0.157768
282238,-0.216691,-0.17306,-0.136701,-0.044522,-0.034536,-0.026909,0.084910,0.088481,0.086917,0.086022,...,0.064757,0.091887,0.093317,0.120140,0.11467,0.121144,0.153011,0.119427,0.125901,0.157768


In [272]:
equity_value_1.iloc[-1][equity_value_1.iloc[-1] > 0].sort_values().tail(20)

(0.0025, -0.0035, -0.015)    0.172220
(0.002, -0.003, -0.02)       0.172978
(0.0025, -0.004, -0.02)      0.175801
(0.0025, -0.004, -0.015)     0.176977
(0.0015, -0.004, -0.02)      0.178862
(0.0015, -0.0035, -0.02)     0.181048
(0.0015, -0.0035, -0.015)    0.181744
(0.002, -0.003, -0.015)      0.184167
(0.0015, -0.003, -0.025)     0.189040
(0.002, -0.003, -0.025)      0.194832
(0.0015, -0.004, -0.025)     0.199602
(0.002, -0.0035, -0.02)      0.200805
(0.0015, -0.0035, -0.025)    0.201735
(0.0025, -0.0035, -0.025)    0.202911
(0.002, -0.004, -0.02)       0.205562
(0.002, -0.0035, -0.015)     0.206950
(0.0025, -0.004, -0.025)     0.207668
(0.002, -0.004, -0.015)      0.211707
(0.002, -0.0035, -0.025)     0.227703
(0.002, -0.004, -0.025)      0.232460
Name: 282239, dtype: float64

In [253]:
for i in equity_value_1.iloc[-1].sort_values(ascending=False).index.to_list():
    print(i)
    performance(equity_value_1[[i]], transaction_history1[i])

(0.002, -0.004, -0.025)
return: 0.23245996103219385
std: 0.0718101575831413
sharp ratio: 3.1647327993825085
trades: 13.0
odds ratio: 0.6153846153846154
mdd: 0.027586956521739092
skewness: -0.5381430922746622
(0.002, -0.0035, -0.025)
return: 0.2277032287075199
std: 0.07084667313468863
sharp ratio: 3.140630588037813
trades: 14.0
odds ratio: 0.5714285714285714
mdd: 0.027586956521739092
skewness: -0.5897041409860616
(0.002, -0.004, -0.015)
return: 0.21170703042250932
std: 0.06816419437134781
sharp ratio: 3.0295528660911253
trades: 13.0
odds ratio: 0.6153846153846154
mdd: 0.03872618836052494
skewness: -0.7140820600470963
(0.0025, -0.004, -0.025)
return: 0.20766760720910554
std: 0.07130940665256943
sharp ratio: 2.839283296740349
trades: 11.0
odds ratio: 0.6363636363636364
mdd: 0.02219658959565568
skewness: -0.5015598214068567
(0.002, -0.0035, -0.015)
return: 0.20695029809783538
std: 0.06728619038405297
sharp ratio: 2.9983908577123253
trades: 14.0
odds ratio: 0.5714285714285714
mdd: 0.0387261

std: 0.0577979670715363
sharp ratio: 2.15878010151262
trades: 28.0
odds ratio: 0.5357142857142857
mdd: 0.07696091323071019
skewness: -0.5637018226095243
(0.001, -0.003, -0.015)
return: 0.12988996027577515
std: 0.059977278714150845
sharp ratio: 2.0789532794584127
trades: 28.0
odds ratio: 0.39285714285714285
mdd: 0.04396265581643515
skewness: -0.38070902774862064
(0.002, -0.002, -0.025)
return: 0.12817328021954083
std: 0.0523931679909235
sharp ratio: 2.3471243472210825
trades: 21.0
odds ratio: 0.5238095238095238
mdd: 0.0777899276245167
skewness: -0.7277980340013629
(0.001, -0.003, -0.02)
return: 0.124690299261561
std: 0.057940611652317446
sharp ratio: 2.0622892277800413
trades: 27.0
odds ratio: 0.4074074074074074
mdd: 0.05423096803570703
skewness: -0.43991545825610867
(0.001, -0.004, -0.02)
return: 0.12075403269183788
std: 0.06009682047097983
sharp ratio: 1.9227977750942382
trades: 21.0
odds ratio: 0.47619047619047616
mdd: 0.07309689958233662
skewness: -0.36221789829686807
(0.0025, -0.00

skewness: -0.3276770383615666
(0.002, -0.001, -0.025)
return: 0.059394009489721844
std: 0.047327549070662826
sharp ratio: 1.1450837948275536
trades: 30.0
odds ratio: 0.4
mdd: 0.12839030545424301
skewness: -0.3865515161200875
(0.004, -0.0025, -0.025)
return: 0.05710663856058251
std: 0.03724838366891493
sharp ratio: 1.3935272741485534
trades: 14.0
odds ratio: 0.42857142857142855
mdd: 0.040616647609533874
skewness: -0.33853347594496974
(0.003, -0.0025, -0.025)
return: 0.05710663856058251
std: 0.03724838366891493
sharp ratio: 1.3935272741485534
trades: 14.0
odds ratio: 0.42857142857142855
mdd: 0.040616647609533874
skewness: -0.33853347594496974
(0.0035, -0.0025, -0.025)
return: 0.05710663856058251
std: 0.03724838366891493
sharp ratio: 1.3935272741485534
trades: 14.0
odds ratio: 0.42857142857142855
mdd: 0.040616647609533874
skewness: -0.33853347594496974
(0.004, -0.0025, -0.02)
return: 0.05188101566943887
std: 0.03694029755395985
sharp ratio: 1.2636881335687795
trades: 15.0
odds ratio: 0.46

(0.001, -0.001, -0.025)
return: -0.14435176434427555
std: 0.04846353618285077
sharp ratio: -3.0858615801377636
trades: 73.0
odds ratio: 0.3013698630136986
mdd: 0.165886647945719
skewness: -0.46974232152940987
(0.001, -0.001, -0.02)
return: -0.1807098899993925
std: 0.05806907081088201
sharp ratio: -3.2015303052611173
trades: 80.0
odds ratio: 0.3125
mdd: 0.1945030972119676
skewness: -0.3690666602220975
(0.001, -0.001, -0.015)
return: -0.2091557029840436
std: 0.06283702812275517
sharp ratio: -3.4112960047265344
trades: 90.0
odds ratio: 0.3
mdd: 0.2187779864280577
skewness: -0.359426620893958


In [254]:
profit_col = [(0.002, -0.004, -0.025), (0.0015, -0.002, -0.015), (0.001, -0.0025, -0.025), (0.001, -0.0025, -0.015), (0.001, -0.002, -0.015), (0.001, -0.0025, -0.02), (0.001, -0.002, -0.02), (0.001, -0.002, -0.025), (0.002, -0.0015, -0.015), (0.002, -0.0015, -0.02), (0.002, -0.001, -0.025), (0.0025, -0.0015, -0.015), (0.002, -0.001, -0.02), (0.002, -0.001, -0.015), (0.0015, -0.0015, -0.02), (0.0015, -0.0015, -0.025), (0.0015, -0.0015, -0.015)]
profit_col

[(0.002, -0.004, -0.025),
 (0.0015, -0.002, -0.015),
 (0.001, -0.0025, -0.025),
 (0.001, -0.0025, -0.015),
 (0.001, -0.002, -0.015),
 (0.001, -0.0025, -0.02),
 (0.001, -0.002, -0.02),
 (0.001, -0.002, -0.025),
 (0.002, -0.0015, -0.015),
 (0.002, -0.0015, -0.02),
 (0.002, -0.001, -0.025),
 (0.0025, -0.0015, -0.015),
 (0.002, -0.001, -0.02),
 (0.002, -0.001, -0.015),
 (0.0015, -0.0015, -0.02),
 (0.0015, -0.0015, -0.025),
 (0.0015, -0.0015, -0.015)]

In [255]:
#adding hard stop 
#main strategy (test set)
#record transaction
transaction_history2 = {}
#record equity values
equity_value_2 = pd.DataFrame()
#record each returns
each_return2 = pd.DataFrame()
#record unrealized gain
unrealized_2 = pd.DataFrame()
for l in range(len(profit_col)):
    df_test['side'] = pd.DataFrame(data = np.zeros((len(df_test), 1)), index = df_test.index[:])
    for j in range(len(df_test)):
        if df_test['pred_return'].iloc[j] > profit_col[l][0]:
            #long side
            df_test['side'].iloc[j] = 1
        elif df_test['pred_return'].iloc[j] < profit_col[l][1]:
            #short side
            df_test['side'].iloc[j] = -1
    date_idx = df_test['TxTime'].unique()
    #紀錄已實現
    equity_value2 = pd.DataFrame()
    #紀錄未實現
    unrealized2 = pd.DataFrame()
    for k in date_idx:
        df = df_test[df_test.TxTime == k]
        #side = 0
        for i in range(len(df)):
            #first tick
            if i == 0:
                df['holding'].iloc[i] = 0
                #ask1 may be 0, which means 漲停 (買不到)
                if (df['side'].iloc[i] == 1) and (df['Ask1'].iloc[i] != 0):
                    #side = 1
                    df['holding'].iloc[i] = 1
                    #stock_price = df['Ask1'].iloc[i]
                    df['equity_value'].iloc[i] = 0
                    transaction_history2 = record_transaction(profit_col[l], k, i, 1, df['Ask1'].iloc[i], 0, transaction_history2)
                elif (df['side'].iloc[i] == 1) and (df['Ask1'].iloc[i] == 0):
                    side = 0
                    df['equity_value'].iloc[i] = 0
                elif df['side'].iloc[i] == -1:
                    #side = -1
                    df['holding'].iloc[i] = -1
                    #stock_price = df['Bid1'].iloc[i]
                    df['equity_value'].iloc[i] = - 0.0015
                    df['unrealized'].iloc[i] = -0.0015
                    transaction_history2 = record_transaction(profit_col[l], k, i, -1, df['Bid1'].iloc[i], -0.0015, transaction_history2)
                else:
                    #side = 0
                    df['holding'].iloc[i] = 0
                    df['equity_value'].iloc[i] = 0
            #last tick
            elif i == (len(df) - 1):
                if df['holding'].iloc[i-1] == 1:
                    df['equity_value'].iloc[i] = (df['Bid1'].iloc[i] / transaction_history2[profit_col[l]][-1]['Price'] - 1) - 0.0015
                    df['unrealized'].iloc[i] = 'realized'
                    #df['unrealized'].iloc[i] = df['Bid1'].iloc[i] / df['mid1'].iloc[i-1] - 1
                    transaction_history2 = record_transaction(profit_col[l], k, i, -1, df['Bid1'].iloc[i], df['equity_value'].iloc[i], transaction_history2)
                #ask1 may be 0, which means 漲停 (買不到)
                elif (df['holding'].iloc[i-1] == -1) and (df['Ask1'].iloc[i] != 0):
                    df['equity_value'].iloc[i] = (df['Ask1'].iloc[i] / transaction_history2[profit_col[l]][-1]['Price'] - 1) * (-1)
                    df['unrealized'].iloc[i] = 'realized'
                    #df['unrealized'].iloc[i] = (df['Ask1'].iloc[i] / df['mid1'].iloc[i-1] - 1) * (-1)
                    transaction_history2 = record_transaction(profit_col[l], k, i, 1, df['Ask1'].iloc[i], df['equity_value'].iloc[i], transaction_history2)
                #收盤如果漲停，空單結不掉，先用mid1結
                elif (df['holding'].iloc[i-1] == -1) and (df['Ask1'].iloc[i] == 0):
                    df['equity_value'].iloc[i] = (df['mid1'].iloc[i] / transaction_history2[profit_col[l]][-1]['Price'] - 1) * (-1)
                    df['unrealized'].iloc[i] = 'realized'
                    #df['unrealized'].iloc[i] = (df['Ask1'].iloc[i] / df['mid1'].iloc[i-1] - 1) * (-1)
                    transaction_history2 = record_transaction(profit_col[l], k, i, 1, df['mid1'].iloc[i], df['equity_value'].iloc[i], transaction_history2)
                else:
                    df['equity_value'].iloc[i] = 0
            else:
                if df['holding'].iloc[i-1] == 1:
                    #hard stop 1 -1 = 0 -> 0
                    if ((df['mid1'].iloc[i-1] / transaction_history2[profit_col[l]][-1]['Price'] - 1) < profit_col[l][2]): #or ((df['mid1'].iloc[i-1] / transaction_history1[paras[l]][-1]['Price'] - 1) > paras[l][2]):
                        df['holding'].iloc[i] = 0
                        df['equity_value'].iloc[i] = (df['Bid1'].iloc[i] / transaction_history2[profit_col[l]][-1]['Price'] - 1) - 0.0015
                        df['unrealized'].iloc[i] = 'realized'
                        #df['unrealized'].iloc[i] = df['Bid1'].iloc[i] / df['mid1'].iloc[i-1] - 1
                        transaction_history2 = record_transaction(profit_col[l], k, i, -1, df['Bid1'].iloc[i], df['equity_value'].iloc[i], transaction_history2)
                    #change side 1 -2 -> -1
                    elif (df['side'].iloc[i] == -1): #and ((df['mid1'].iloc[i-1] / transaction_history1[paras[l]][-1]['Price'] - 1) > 0.01):
                        df['holding'].iloc[i] = -1
                        df['equity_value'].iloc[i] = (df['Bid1'].iloc[i] / transaction_history2[profit_col[l]][-1]['Price'] - 1) - (0.0015 * 2)
                        df['unrealized'].iloc[i] = 'realized'
                        transaction_history2 = record_transaction(profit_col[l], k, i, -1, df['Bid1'].iloc[i], df['equity_value'].iloc[i] + 0.0015, transaction_history2)
                        transaction_history2 = record_transaction(profit_col[l], k, i, -1, df['Bid1'].iloc[i], -0.0015, transaction_history2)
                    else:
                        df['holding'].iloc[i] = df['holding'].iloc[i-1]
                        df['equity_value'].iloc[i] = 0
                        df['unrealized'].iloc[i] = (df['mid1'].iloc[i] / df['mid1'].iloc[i-1] - 1)
                elif df['holding'].iloc[i-1] == -1:
                    #hard stop -1 +1 -> 0
                    if (df['Ask1'].iloc[i] != 0) and ((df['mid1'].iloc[i-1] / transaction_history2[profit_col[l]][-1]['Price'] - 1) * (-1) < profit_col[l][2]): #or ((df['mid1'].iloc[i-1] / transaction_history1[paras[l]][-1]['Price'] - 1) * (-1) > paras[l][2])):
                        df['holding'].iloc[i] = 0
                        df['equity_value'].iloc[i] = (df['Ask1'].iloc[i] / transaction_history2[profit_col[l]][-1]['Price'] - 1) * (-1)
                        df['unrealized'].iloc[i] = 'realized'
                        #df['unrealized'].iloc[i] = (df['Ask1'].iloc[i] / df['mid1'].iloc[i-1] - 1) * (-1)
                        transaction_history2 = record_transaction(profit_col[l], k, i, 1, df['Ask1'].iloc[i], df['equity_value'].iloc[i], transaction_history2)
                    #change side -1 +2 -> 1
                    elif (df['Ask1'].iloc[i] != 0) and (df['side'].iloc[i] == 1): #and ((df['mid1'].iloc[i-1] / transaction_history1[paras[l]][-1]['Price'] - 1) * (-1) > 0.01):
                        df['holding'].iloc[i] = 1
                        df['equity_value'].iloc[i] = (df['Ask1'].iloc[i] / transaction_history2[profit_col[l]][-1]['Price'] - 1) * (-1)
                        df['unrealized'].iloc[i] = 'realized'
                        #df['unrealized'].iloc[i] = (df['Ask1'].iloc[i] / df['mid1'].iloc[i-1] - 1) * (-1)
                        transaction_history2 = record_transaction(profit_col[l], k, i, 1, df['Ask1'].iloc[i], df['equity_value'].iloc[i], transaction_history2)
                        transaction_history2 = record_transaction(profit_col[l], k, i, 1, df['Ask1'].iloc[i], 0, transaction_history2)
                    else:
                        df['holding'].iloc[i] = df['holding'].iloc[i-1]
                        df['equity_value'].iloc[i] = 0
                        df['unrealized'].iloc[i] = (df['mid1'].iloc[i] / df['mid1'].iloc[i-1] - 1) * (-1)
                elif df['holding'].iloc[i-1] == 0:
                    if (df['side'].iloc[i] == 1) and (df['Ask1'].iloc[i] != 0):
                        df['holding'].iloc[i] = 1
                        df['equity_value'].iloc[i] = 0
                        transaction_history2 = record_transaction(profit_col[l], k, i, 1, df['Ask1'].iloc[i], 0, transaction_history2)
                    elif df['side'].iloc[i] == -1:
                        df['holding'].iloc[i] = -1
                        df['equity_value'].iloc[i] = - 0.0015
                        df['unrealized'].iloc[i] = -0.0015
                        transaction_history2 = record_transaction(profit_col[l], k, i, -1, df['Bid1'].iloc[i], -0.0015, transaction_history2)
                    else:
                        df['holding'].iloc[i] = 0
                        df['equity_value'].iloc[i] = 0
        equity_value2 = pd.concat([equity_value2, df[['equity_value']]])
        unrealized2 = pd.concat([unrealized2, df['unrealized']])
    #concat
    equity_value_2 = pd.concat([equity_value_2, np.cumsum(equity_value2)], axis = 1)
    each_return2 = pd.concat([each_return2, equity_value2], axis = 1)
    unrealized_2 = pd.concat([unrealized_2, unrealized2], axis = 1)

C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\U

In [259]:
equity_value_2.columns = profit_col
equity_value_2

,"(0.002, -0.004, -0.025)","(0.0015, -0.002, -0.015)","(0.001, -0.0025, -0.025)","(0.001, -0.0025, -0.015)","(0.001, -0.002, -0.015)","(0.001, -0.0025, -0.02)","(0.001, -0.002, -0.02)","(0.001, -0.002, -0.025)","(0.002, -0.0015, -0.015)","(0.002, -0.0015, -0.02)","(0.002, -0.001, -0.025)","(0.0025, -0.0015, -0.015)","(0.002, -0.001, -0.02)","(0.002, -0.001, -0.015)","(0.0015, -0.0015, -0.02)","(0.0015, -0.0015, -0.025)","(0.0015, -0.0015, -0.015)"
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104305,0.039822,0.022195,0.017846,0.014273,0.022195,0.017951,0.019066,0.014411,0.014036,0.017649,0.019634,0.014036,0.017680,0.012974,0.017649,0.017460,0.014036
104306,0.039822,0.022195,0.017846,0.014273,0.022195,0.017951,0.019066,0.014411,0.014036,0.017649,0.019634,0.014036,0.017680,0.012974,0.017649,0.017460,0.014036
104307,0.039822,0.022195,0.017846,0.014273,0.022195,0.017951,0.019066,0.014411,0.014036,0.017649,0.019634,0.014036,0.017680,0.012974,0.017649,0.017460,0.014036
104308,0.039822,0.022195,0.017846,0.014273,0.022195,0.017951,0.019066,0.014411,0.014036,0.017649,0.019634,0.014036,0.017680,0.012974,0.017649,0.017460,0.014036


In [266]:
for i in equity_value_2.iloc[-1].sort_values(ascending = False).index.tolist():
    print(i)
    performance(equity_value_2[[i]], transaction_history2[i])

(0.002, -0.004, -0.025)
return: 0.03982231404958675
std: 0.0188681881375156
sharp ratio: 1.8349570079146729
trades: 1.0
odds ratio: 1.0
mdd: 0.0015
skewness: -0.817343570275094
(0.0015, -0.002, -0.015)
return: 0.030016275649718982
std: 0.02523345655467455
sharp ratio: 0.9834671518722914
trades: 8.0
odds ratio: 0.625
mdd: 0.037004578109592956
skewness: -0.4091385636839937
(0.001, -0.002, -0.015)
return: 0.030016275649718982
std: 0.02523345655467455
sharp ratio: 0.9834671518722914
trades: 8.0
odds ratio: 0.625
mdd: 0.037004578109592956
skewness: -0.4091385636839937
(0.001, -0.002, -0.02)
return: 0.026886778563329716
std: 0.024553751650204143
sharp ratio: 0.8832368622230244
trades: 7.0
odds ratio: 0.5714285714285714
mdd: 0.03380766857110801
skewness: -0.3097342426698017
(0.002, -0.001, -0.025)
return: 0.026345696270801855
std: 0.023801137715416167
sharp ratio: 0.888432163354344
trades: 8.0
odds ratio: 0.625
mdd: 0.034993900520815135
skewness: -0.36977135674088285
(0.002, -0.001, -0.02)
re

In [3]:
#using MA
df_train = pd.read_csv('./data/df_train_MA.csv')
df_test = pd.read_csv('./data/df_test_MA.csv')

In [8]:
df_train = df_train.drop(['Unnamed: 0'], axis = 1)
df_train

,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,...,dbq3_dt_10,dbq4_dt_10,dbq5_dt_10,mid1,equity_value,unrealized,holding,pred_mid,MA,MA_side
0,398.0,1,6,396.0,398.5,2,40,395.0,399.0,144,...,-31.0,-8.0,5.0,397.25,0.0,0.0,0.0,398.48914,398.059927,0.001078
1,398.5,1,6,396.0,398.5,1,40,395.0,399.0,144,...,-31.0,-8.0,5.0,397.25,0.0,0.0,0.0,398.20010,398.552942,-0.000885
2,0.0,0,20,396.0,398.5,1,40,395.0,399.0,144,...,0.0,0.0,0.0,397.25,0.0,0.0,0.0,397.96420,398.394876,-0.001081
3,0.0,0,20,396.0,398.5,1,40,395.0,399.0,144,...,0.0,0.0,0.0,397.25,0.0,0.0,0.0,397.70130,398.279758,-0.001452
4,0.0,0,20,396.0,398.0,1,40,395.0,398.5,1,...,0.0,0.0,0.0,397.00,0.0,0.0,0.0,397.67670,398.104349,-0.001074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282226,0.0,0,16,460.5,461.0,22,16,460.0,461.5,11,...,0.0,0.0,0.0,460.75,0.0,0.0,0.0,461.32422,461.318427,0.000013
282227,0.0,0,16,460.5,461.0,21,16,460.0,461.5,11,...,0.0,0.0,0.0,460.75,0.0,0.0,0.0,461.38740,461.335544,0.000112
282228,460.5,1,15,460.5,461.0,21,16,460.0,461.5,11,...,0.0,0.0,0.0,460.75,0.0,0.0,0.0,461.41562,461.356308,0.000129
282229,461.0,1,15,460.5,461.0,20,16,460.0,461.5,11,...,0.0,0.0,0.0,460.75,0.0,0.0,0.0,461.40878,461.366208,0.000092


In [9]:
df_test = df_test.drop(['Unnamed: 0'], axis = 1)
df_test

,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,...,dbq3_dt_10,dbq4_dt_10,dbq5_dt_10,mid1,equity_value,unrealized,holding,pred_mid,MA,MA_side
0,458.0,2,10,457.0,458.5,5,14,456.5,459.0,10,...,27.0,-38.0,11.0,457.75,0.0,0.0,0.0,458.48956,459.195993,-0.001538
1,0.0,0,11,457.0,458.5,5,14,456.5,459.0,10,...,27.0,-38.0,11.0,457.75,0.0,0.0,0.0,458.57654,459.133038,-0.001212
2,0.0,0,1,458.0,458.5,5,11,457.0,459.0,10,...,0.0,0.0,0.0,458.25,0.0,0.0,0.0,458.60715,459.065320,-0.000998
3,458.0,1,11,457.0,458.5,5,14,456.5,459.0,10,...,27.0,-38.0,11.0,457.75,0.0,0.0,0.0,458.74290,458.985858,-0.000529
4,0.0,0,11,457.0,457.5,1,14,456.5,458.5,5,...,27.0,-38.0,11.0,457.25,0.0,0.0,0.0,458.66193,458.920044,-0.000562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104294,443.5,2,8,443.5,444.0,8,24,443.0,444.5,23,...,-9.0,10.0,-11.0,443.75,0.0,0.0,0.0,444.41263,444.305936,0.000240
104295,443.5,1,7,443.5,444.0,8,24,443.0,444.5,23,...,-9.0,11.0,-11.0,443.75,0.0,0.0,0.0,444.41953,444.329340,0.000203
104296,0.0,0,7,443.5,444.0,8,24,443.0,444.5,23,...,0.0,0.0,0.0,443.75,0.0,0.0,0.0,444.42450,444.346219,0.000176
104297,443.5,3,4,443.5,444.0,8,24,443.0,444.5,23,...,0.0,0.0,0.0,443.75,0.0,0.0,0.0,444.45334,444.364877,0.000199


In [24]:
#setting parameters
import itertools 
long_side, short_side, stop_1, stop_2 = [0.002, 0.0017, 0.0015, 0.0013, 0.001], [-0.002, -0.0017, -0.0015, -0.0013, -0.001], [0.02], [-0.01]
paras = list(itertools.product(long_side, short_side, stop_1, stop_2))

In [25]:
#adding hard stop 
#main strategy (train set)
#record transaction
transaction_history1 = {}
#record equity values
equity_value_1 = pd.DataFrame()
#record each returns
each_return1 = pd.DataFrame()
#record unrealized gain
unrealized_1 = pd.DataFrame()
for l in range(len(paras)):
    df_train['side'] = pd.DataFrame(data = np.zeros((len(df_train), 1)), index = df_train.index[:])
    for j in range(len(df_train)):
        if df_train['MA_side'].iloc[j] > paras[l][0]:
            #反過來做 1 -> -1
            df_train['side'].iloc[j] = -1
        elif df_train['MA_side'].iloc[j] < paras[l][1]:
            #反過來做 -1 -> 1
            df_train['side'].iloc[j] = 1
    date_idx = df_train['TxTime'].unique()
    #紀錄已實現
    equity_value1 = pd.DataFrame()
    #紀錄未實現
    unrealized1 = pd.DataFrame()
    for k in date_idx:
        df = df_train[df_train.TxTime == k]
        #side = 0
        for i in range(len(df)):
            #first tick
            if i == 0:
                #ask1 may be 0, which means 漲停 (買不到)
                if (df['side'].iloc[i] == 1) and (df['Ask1'].iloc[i] != 0):
                    #side = 1
                    df['holding'].iloc[i] = 1
                    #stock_price = df['Ask1'].iloc[i]
                    df['equity_value'].iloc[i] = 0
                    transaction_history1 = record_transaction(paras[l], k, i, 1, df['Ask1'].iloc[i], 0, transaction_history1)
                elif (df['side'].iloc[i] == 1) and (df['Ask1'].iloc[i] == 0):
                    side = 0
                    df['equity_value'].iloc[i] = 0
                elif df['side'].iloc[i] == -1:
                    #side = -1
                    df['holding'].iloc[i] = -1
                    #stock_price = df['Bid1'].iloc[i]
                    df['equity_value'].iloc[i] = - 0.0015
                    df['unrealized'].iloc[i] = -0.0015
                    transaction_history1 = record_transaction(paras[l], k, i, -1, df['Bid1'].iloc[i], -0.0015, transaction_history1)
                else:
                    #side = 0
                    df['holding'].iloc[i] = 0
                    df['equity_value'].iloc[i] = 0
            #last tick
            elif i == (len(df) - 1):
                if df['holding'].iloc[i-1] == 1:
                    df['equity_value'].iloc[i] = (df['Bid1'].iloc[i] / transaction_history1[paras[l]][-1]['Price'] - 1) - 0.0015
                    df['unrealized'].iloc[i] = 'realized'
                    #df['unrealized'].iloc[i] = df['Bid1'].iloc[i] / df['mid1'].iloc[i-1] - 1
                    transaction_history1 = record_transaction(paras[l], k, i, -1, df['Bid1'].iloc[i], df['equity_value'].iloc[i], transaction_history1)
                #ask1 may be 0, which means 漲停 (買不到)
                elif (df['holding'].iloc[i-1] == -1) and (df['Ask1'].iloc[i] != 0):
                    df['equity_value'].iloc[i] = (df['Ask1'].iloc[i] / transaction_history1[paras[l]][-1]['Price'] - 1) * (-1)
                    df['unrealized'].iloc[i] = 'realized'
                    #df['unrealized'].iloc[i] = (df['Ask1'].iloc[i] / df['mid1'].iloc[i-1] - 1) * (-1)
                    transaction_history1 = record_transaction(paras[l], k, i, 1, df['Ask1'].iloc[i], df['equity_value'].iloc[i], transaction_history1)
                #收盤如果漲停，空單結不掉，先用mid1結
                elif (df['holding'].iloc[i-1] == -1) and (df['Ask1'].iloc[i] == 0):
                    df['equity_value'].iloc[i] = (df['mid1'].iloc[i] / transaction_history1[paras[l]][-1]['Price'] - 1) * (-1)
                    df['unrealized'].iloc[i] = 'realized'
                    #df['unrealized'].iloc[i] = (df['Ask1'].iloc[i] / df['mid1'].iloc[i-1] - 1) * (-1)
                    transaction_history1 = record_transaction(paras[l], k, i, 1, df['mid1'].iloc[i], df['equity_value'].iloc[i], transaction_history1)
                else:
                    df['equity_value'].iloc[i] = 0
            else:
                if df['holding'].iloc[i-1] == 1:
                    #hard stop 1 -1 = 0 -> 0
                    if ((df['mid1'].iloc[i-1] / transaction_history1[paras[l]][-1]['Price'] - 1) < paras[l][3]) or ((df['mid1'].iloc[i-1] / transaction_history1[paras[l]][-1]['Price'] - 1) > paras[l][2]):
                        df['holding'].iloc[i] = 0
                        df['equity_value'].iloc[i] = (df['Bid1'].iloc[i] / transaction_history1[paras[l]][-1]['Price'] - 1) - 0.0015
                        df['unrealized'].iloc[i] = 'realized'
                        #df['unrealized'].iloc[i] = df['Bid1'].iloc[i] / df['mid1'].iloc[i-1] - 1
                        transaction_history1 = record_transaction(paras[l], k, i, -1, df['Bid1'].iloc[i], df['equity_value'].iloc[i], transaction_history1)
                    #change side 1 -2 -> -1
                    elif (df['side'].iloc[i] == -1) and ((df['mid1'].iloc[i-1] / transaction_history1[paras[l]][-1]['Price'] - 1) > 0.01):
                        df['holding'].iloc[i] = -1
                        df['equity_value'].iloc[i] = (df['Bid1'].iloc[i] / transaction_history1[paras[l]][-1]['Price'] - 1) - (0.0015 * 2)
                        df['unrealized'].iloc[i] = 'realized'
                        transaction_history1 = record_transaction(paras[l], k, i, -1, df['Bid1'].iloc[i], df['equity_value'].iloc[i] + 0.0015, transaction_history1)
                        transaction_history1 = record_transaction(paras[l], k, i, -1, df['Bid1'].iloc[i], -0.0015, transaction_history1)
                    else:
                        df['holding'].iloc[i] = df['holding'].iloc[i-1]
                        df['equity_value'].iloc[i] = 0
                        df['unrealized'].iloc[i] = (df['mid1'].iloc[i] / df['mid1'].iloc[i-1] - 1)
                elif df['holding'].iloc[i-1] == -1:
                    #hard stop -1 +1 -> 0
                    if (df['Ask1'].iloc[i] != 0) and (((df['mid1'].iloc[i-1] / transaction_history1[paras[l]][-1]['Price'] - 1) * (-1) < paras[l][3]) or ((df['mid1'].iloc[i-1] / transaction_history1[paras[l]][-1]['Price'] - 1) * (-1) > paras[l][2])):
                        df['holding'].iloc[i] = 0
                        df['equity_value'].iloc[i] = (df['Ask1'].iloc[i] / transaction_history1[paras[l]][-1]['Price'] - 1) * (-1)
                        df['unrealized'].iloc[i] = 'realized'
                        #df['unrealized'].iloc[i] = (df['Ask1'].iloc[i] / df['mid1'].iloc[i-1] - 1) * (-1)
                        transaction_history1 = record_transaction(paras[l], k, i, 1, df['Ask1'].iloc[i], df['equity_value'].iloc[i], transaction_history1)
                    #change side -1 +2 -> 1
                    elif (df['Ask1'].iloc[i] != 0) and (df['side'].iloc[i] == 1) and ((df['mid1'].iloc[i-1] / transaction_history1[paras[l]][-1]['Price'] - 1) * (-1) > 0.01):
                        df['holding'].iloc[i] = 1
                        df['equity_value'].iloc[i] = (df['Ask1'].iloc[i] / transaction_history1[paras[l]][-1]['Price'] - 1) * (-1)
                        df['unrealized'].iloc[i] = 'realized'
                        #df['unrealized'].iloc[i] = (df['Ask1'].iloc[i] / df['mid1'].iloc[i-1] - 1) * (-1)
                        transaction_history1 = record_transaction(paras[l], k, i, 1, df['Ask1'].iloc[i], df['equity_value'].iloc[i], transaction_history1)
                        transaction_history1 = record_transaction(paras[l], k, i, 1, df['Ask1'].iloc[i], 0, transaction_history1)
                    else:
                        df['holding'].iloc[i] = df['holding'].iloc[i-1]
                        df['equity_value'].iloc[i] = 0
                        df['unrealized'].iloc[i] = (df['mid1'].iloc[i] / df['mid1'].iloc[i-1] - 1) * (-1)
                elif df['holding'].iloc[i-1] == 0:
                    if (df['side'].iloc[i] == 1) and (df['Ask1'].iloc[i] != 0):
                        df['holding'].iloc[i] = 1
                        df['equity_value'].iloc[i] = 0
                        transaction_history1 = record_transaction(paras[l], k, i, 1, df['Ask1'].iloc[i], 0, transaction_history1)
                    elif df['side'].iloc[i] == -1:
                        df['holding'].iloc[i] = -1
                        df['equity_value'].iloc[i] = - 0.0015
                        df['unrealized'].iloc[i] = -0.0015
                        transaction_history1 = record_transaction(paras[l], k, i, -1, df['Bid1'].iloc[i], -0.0015, transaction_history1)
                    else:
                        df['holding'].iloc[i] = 0
                        df['equity_value'].iloc[i] = 0
        equity_value1 = pd.concat([equity_value1, df[['equity_value']]])
        unrealized1 = pd.concat([unrealized1, df['unrealized']])
    #concat
    equity_value_1 = pd.concat([equity_value_1, np.cumsum(equity_value1)], axis = 1)
    each_return1 = pd.concat([each_return1, equity_value1], axis = 1)
    unrealized_1 = pd.concat([unrealized_1, unrealized1], axis = 1)

C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\U

C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\user10\Anaconda3\envs\PythonGPU\lib\site-pa

In [27]:
equity_value_1.iloc[-1]

equity_value   -0.133258
equity_value   -0.119373
equity_value   -0.131006
equity_value   -0.154348
equity_value   -0.244622
equity_value   -0.140385
equity_value   -0.158204
equity_value   -0.202256
equity_value   -0.243593
equity_value   -0.291155
equity_value   -0.146198
equity_value   -0.157851
equity_value   -0.167473
equity_value   -0.230638
equity_value   -0.236158
equity_value   -0.082392
equity_value   -0.139127
equity_value   -0.123261
equity_value   -0.145783
equity_value   -0.154240
equity_value   -0.158560
equity_value   -0.184146
equity_value   -0.215070
equity_value   -0.340762
equity_value   -0.209147
Name: 282230, dtype: float64

In [14]:
equity_value_1.columns = paras
equity_value_1.iloc[-1]

(0.002, -0.002, 0.02, -0.01)     -0.100921
(0.002, -0.0017, 0.02, -0.01)    -0.142712
(0.002, -0.0015, 0.02, -0.01)    -0.203508
(0.002, -0.0013, 0.02, -0.01)    -0.342304
(0.002, -0.001, 0.02, -0.01)     -0.318016
(0.0017, -0.002, 0.02, -0.01)    -0.164444
(0.0017, -0.0017, 0.02, -0.01)   -0.196005
(0.0017, -0.0015, 0.02, -0.01)   -0.204657
(0.0017, -0.0013, 0.02, -0.01)   -0.282664
(0.0017, -0.001, 0.02, -0.01)    -0.312003
(0.0015, -0.002, 0.02, -0.01)    -0.244278
(0.0015, -0.0017, 0.02, -0.01)   -0.270741
(0.0015, -0.0015, 0.02, -0.01)   -0.293752
(0.0015, -0.0013, 0.02, -0.01)   -0.412514
(0.0015, -0.001, 0.02, -0.01)    -0.421063
(0.0013, -0.002, 0.02, -0.01)    -0.358915
(0.0013, -0.0017, 0.02, -0.01)   -0.320583
(0.0013, -0.0015, 0.02, -0.01)   -0.279946
(0.0013, -0.0013, 0.02, -0.01)   -0.424178
(0.0013, -0.001, 0.02, -0.01)    -0.576081
(0.001, -0.002, 0.02, -0.01)     -0.265309
(0.001, -0.0017, 0.02, -0.01)    -0.225488
(0.001, -0.0015, 0.02, -0.01)    -0.122741
(0.001, -0.